In [1]:
import openai

from fp_dataset_artifacts.utils import init_openai
from fp_dataset_artifacts.boolq import format_question, format_answer
from datasets import list_datasets, load_dataset, list_metrics, load_metric

init_openai()

data = load_dataset('boolq')
data

Using custom data configuration default
Reusing dataset boolq (/home/x/.cache/huggingface/datasets/boolq/default/0.1.0/bf0dd57da941c50de94ae3ce3cef7fea48c08f337a4b7aac484e9dddc5aa24e5)


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['question', 'answer', 'passage'],
        num_rows: 9427
    })
    validation: Dataset({
        features: ['question', 'answer', 'passage'],
        num_rows: 3270
    })
})

In [43]:
def map_finetune_train(x):
    question = format_question(x['question'])
    passage = x['passage']
    answer = format_answer(x['answer'])

    return {
        'prompt': f"{passage}\n\nQ: {question}\n\nA: ",
        'completion': answer,
    }

In [38]:
train = data['train'].map(map_finetune_train)
train = train.remove_columns(['question', 'answer', 'passage'])

Loading cached processed dataset at /home/x/.cache/huggingface/datasets/boolq/default/0.1.0/bf0dd57da941c50de94ae3ce3cef7fea48c08f337a4b7aac484e9dddc5aa24e5/cache-5ebed985e83a6877.arrow


In [51]:
train[0]

{'prompt': 'Persian (/ˈpɜːrʒən, -ʃən/), also known by its endonym Farsi (فارسی fārsi (fɒːɾˈsiː) ( listen)), is one of the Western Iranian languages within the Indo-Iranian branch of the Indo-European language family. It is primarily spoken in Iran, Afghanistan (officially known as Dari since 1958), and Tajikistan (officially known as Tajiki since the Soviet era), and some other regions which historically were Persianate societies and considered part of Greater Iran. It is written in the Persian alphabet, a modified variant of the Arabic script, which itself evolved from the Aramaic alphabet.\n\nQ: Do iran and afghanistan speak the same language?\n\nA: ',
 'completion': 'Yes'}

In [41]:
train.to_json('boolq_finetune_train.jsonl')

Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

6223009

In [44]:
valid = data['validation'].map(map_finetune_train)
valid = valid.remove_columns(['question', 'answer', 'passage'])
valid.to_json('boolq_finetune_valid.jsonl')

  0%|          | 0/3270 [00:00<?, ?ex/s]

Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

2134729

In [45]:
openai.File.create(file=open('boolq_finetune_train.jsonl'), purpose='fine-tune')

<File file id=file-gJIlaEZ4vKpcMs8kOd8Tzifx at 0x7f1cfc7172c0> JSON: {
  "bytes": 6223009,
  "created_at": 1638669386,
  "filename": "boolq_finetune_train.jsonl",
  "id": "file-gJIlaEZ4vKpcMs8kOd8Tzifx",
  "object": "file",
  "purpose": "fine-tune",
  "status": "uploaded",
  "status_details": null
}

In [46]:
openai.File.create(file=open('boolq_finetune_valid.jsonl'), purpose='fine-tune')

<File file id=file-MahHYpnc6hjHw2EZQ1CH3jOw at 0x7f1cfc735a90> JSON: {
  "bytes": 2134729,
  "created_at": 1638669392,
  "filename": "boolq_finetune_valid.jsonl",
  "id": "file-MahHYpnc6hjHw2EZQ1CH3jOw",
  "object": "file",
  "purpose": "fine-tune",
  "status": "uploaded",
  "status_details": null
}

In [48]:
openai.FineTune.create(
    training_file='file-gJIlaEZ4vKpcMs8kOd8Tzifx',
    validation_file='file-MahHYpnc6hjHw2EZQ1CH3jOw',
    model='curie',
    n_epochs=4,
    compute_classification_metrics=True,
    classification_positive_class='Yes',
    classification_n_classes=2
)

<FineTune fine-tune id=ft-9ZmeSNYkAUhZEmXkoiigSMdJ at 0x7f1cfc717a40> JSON: {
  "created_at": 1638669489,
  "events": [
    {
      "created_at": 1638669489,
      "level": "info",
      "message": "Created fine-tune: ft-9ZmeSNYkAUhZEmXkoiigSMdJ",
      "object": "fine-tune-event"
    }
  ],
  "fine_tuned_model": null,
  "hyperparams": {
    "batch_size": null,
    "classification_n_classes": 2,
    "classification_positive_class": "Yes",
    "compute_classification_metrics": true,
    "learning_rate_multiplier": null,
    "n_epochs": 4,
    "prompt_loss_weight": 0.1,
    "use_packing": null
  },
  "id": "ft-9ZmeSNYkAUhZEmXkoiigSMdJ",
  "model": "curie",
  "object": "fine-tune",
  "organization_id": "org-5AE307Eg4rc5EAoEA2S2bwkH",
  "result_files": [],
  "status": "pending",
  "training_files": [
    {
      "bytes": 6223009,
      "created_at": 1638669386,
      "filename": "boolq_finetune_train.jsonl",
      "id": "file-gJIlaEZ4vKpcMs8kOd8Tzifx",
      "object": "file",
      "purpos

In [52]:
openai.FineTune.retrieve('ft-9ZmeSNYkAUhZEmXkoiigSMdJ')

<FineTune fine-tune id=ft-9ZmeSNYkAUhZEmXkoiigSMdJ at 0x7f1cfc735a40> JSON: {
  "created_at": 1638669489,
  "events": [
    {
      "created_at": 1638669489,
      "level": "info",
      "message": "Created fine-tune: ft-9ZmeSNYkAUhZEmXkoiigSMdJ",
      "object": "fine-tune-event"
    },
    {
      "created_at": 1638669494,
      "level": "info",
      "message": "Fine-tune enqueued. Queue number: 0",
      "object": "fine-tune-event"
    },
    {
      "created_at": 1638669499,
      "level": "info",
      "message": "Fine-tune started",
      "object": "fine-tune-event"
    },
    {
      "created_at": 1638669912,
      "level": "info",
      "message": "Completed epoch 1/4",
      "object": "fine-tune-event"
    }
  ],
  "fine_tuned_model": null,
  "hyperparams": {
    "batch_size": 1,
    "classification_n_classes": 2,
    "classification_positive_class": "Yes",
    "compute_classification_metrics": true,
    "learning_rate_multiplier": 0.1,
    "n_epochs": 4,
    "prompt_loss_wei